In [44]:
import minsearch

In [45]:
import json

In [46]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [47]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict ['course']
        documents.append(doc)

In [48]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [49]:
index = minsearch.Index(
    text_fields = ["section","text","question"],
    keyword_fields = ['course']
)

In [50]:
q = 'the course has already started. can i still enrol?'

In [51]:
index.fit(documents)

In [52]:
import os
from huggingface_hub import InferenceClient

In [53]:
client = InferenceClient()

In [54]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict= {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    return results

In [55]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ question. 
Use only the facts from the CONTEXT when answering the question.


QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question =query, context=context).strip()
    return prompt

In [56]:
def llm(prompt):
    response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3-0324",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

    return response.choices[0].message.content

In [57]:
query = 'the course just started, how do i enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [58]:
rag (query)

'To enroll in the course, follow these steps from the CONTEXT:\n\n1. **Register before the course starts** using the provided link (mentioned in the "Course - When will the course start?" section).  \n2. **Subscribe to the course public Google Calendar** (Desktop only) to stay updated.  \n3. **Join the course Telegram channel** for announcements.  \n4. **Register in DataTalks.Club\'s Slack** and join the channel (also mentioned in the "Course - When will the course start?" section).  \n\nThe course has already started (as of 15th Jan 2024 at 17h00), but you can still enroll by completing these steps.'

In [59]:
rag('how do i register for the course')

'To register for the course, use the registration link provided in the answer to the question "Course - When will the course start?" The exact steps are:\n\n1. Register before the course starts using this link (the specific link is mentioned in the context but not provided here).  \n2. Join the course Telegram channel for announcements.  \n3. Register in DataTalks.Club\'s Slack and join the channel.  \n\nThe course starts on **15th Jan 2024 at 17h00**.  \n\n(Note: The actual registration link is not included in the context, so you would need to refer to the original source for it.)'

In [60]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [61]:
from elasticsearch import Elasticsearch

In [64]:
es_client = Elasticsearch(
    "http://localhost:9200",
)

In [65]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-question"

es_client.indices.create(index=index_name, body= index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-question'})

In [66]:
import elasticsearch
print(elasticsearch.__version__)


(8, 19, 0)


In [67]:
from tqdm.auto import tqdm

In [68]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [80]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)

    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [81]:
elastic_search(query)

[{'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou will probably also create your own repositories that host your notes, versions of your file, to do this. Here is a great tutorial that shows you how to do 

In [82]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [83]:
rag(query)

"Based on the provided **CONTEXT**, there is no specific information about how to enroll in the course after it has started. However, the FAQ does mention that you can still participate even if you haven't registered, as you remain eligible to submit homework.  \n\nIf you're looking to formally enroll, you may need to check the course platform or contact the course administrators directly, as this detail is not covered in the given FAQ sections.  \n\nFor now, you can:  \n1. **Set up dependencies** (Google Cloud, Python, Terraform, Git).  \n2. **Clone the course repository** to access materials.  \n3. **Follow deadlines** for assignments and final projects.  \n\nIf enrollment is required for additional benefits (e.g., certificates, instructor feedback), further guidance outside this FAQ may be needed."